In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from hansolo import *
import psutil
from sklearn import datasets
from sklearn.model_selection import train_test_split

#retrieve dataset
digits = datasets.load_digits()
images=digits.images/16 #(1797,8,8)
target=digits.target #(1797,)

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
M_train = np.shape(X_train)[0]
M_test = np.shape(X_test)[0]

# Reshape images and compute probabilities
flattened_images_train = X_train.reshape((M_train, -1))
flattened_images_test = X_test.reshape((M_test, -1))

M_out = M_train
M_tot = M_out + M_test

N = 2000 #number of timesteps
p = 1  # spiking proba input neurons 
nu = 0.5 # bias

K_input = 64 #number of input neurons

K_output = 10  #number of output neurons

para_PWA = 2 #parameter expert PWA

expert2 = 'PWA'


eta_output = 0.0005 #parameter EWA output layer


images_out = flattened_images_train /16
images_test = flattened_images_test /16

target_out = y_train
target_test = y_test
Nb_simu = 100 #number of simulations

accuracy = np.zeros((Nb_simu))

for nb in range(Nb_simu):
#Simulation input neurons
    prob_input = np.transpose(np.array([p*images_out for i in range(N)]), axes = (2,0,1))
    input_neurons_out = stats.bernoulli.rvs(prob_input)
    prob_input = np.transpose(np.array([p*images_test for i in range(N)]), axes = (2,0,1))
    input_neurons_test = stats.bernoulli.rvs(prob_input) #to test
    P_input = np.sum(input_neurons_out, axis = 1) / N

#parameters output layer
 
    cum_gain_output = np.zeros((K_output))
    cum_gain_mid2_to_output = np.zeros((K_output, K_input))

    W_output = np.zeros((K_output, K_input, M_out))
    W_output_not_renorm = np.zeros((K_output, K_input, M_out))
    W_output[:, :, 0] += 1/K_input
    W_output_not_renorm[:, :, 0] += 1

#training output layer
    for m in range(M_out-1):
        gain_output = GainOutput(K_output, K_input, P_input[:,m], target_out[m]) 
        cum_gain_mid2_to_output += gain_output
        cum_gain_output += np.sum(W_output[:, :, m] * gain_output, axis = 1)
        
        if expert2 == "EWA":
            W_output_not_renorm[ :, :, m+1] = EWA(W_output_not_renorm[:, :, m], eta_output, gain_output)
            W_output[:,:, m+1] = (W_output_not_renorm[:, :, m+1].T / np.sum(W_output_not_renorm[:, :, m + 1], axis = 1)).T

        else:
            W_output_not_renorm[:, :, m+1] = PWA(para_PWA, K_output, K_input, cum_gain_output, cum_gain_mid2_to_output)
            not_zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) != 0)[0]
            zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) == 0)[0]
            W_output[not_zero,:, m + 1] = (
                W_output_not_renorm[not_zero, :, m + 1].T / np.sum(W_output_not_renorm[not_zero,:, m + 1], axis=1)
                ).T
            W_output[zero,:, m+1] = 1/K_input

#test with new images
    
    answers_test = np.zeros((M_test))
    for m in range(M_test):
     
    # Simulation output neurons (without Kalikow)
        probas_output = np.sum(W_output[:,None,:,- 1] * input_neurons_test[:,:,m].T, axis = 2)
        probas_output = np.clip(probas_output, 0, 1)
        output_neurons = stats.bernoulli.rvs(probas_output)

        P_output = np.sum(output_neurons, axis = 1) / N
        chosen_class = np.where(P_output == np.max(P_output))[0]

        if chosen_class[0] == target_test[m]:
            answers_test[m] = 1
    accuracy[nb] = np.sum(answers_test)/M_test
    print(nb, accuracy[nb])

#np.save(f"accuracy_{expert2}_{eta_output}_100", accuracy)

